# Pre-processing

This notebook performs the loading, cleaning and initial preparation of the data for analysis and modeling.

### Imports

In [1]:
import pandas as pd
import numpy as np
import os
import json
from pathlib import Path
import re

### Load Data

In [2]:
DATA_DIR = Path("../data/raw")

files = {
    "candidates": DATA_DIR / "candidates.json",
    "vacancies": DATA_DIR / "vacancies.json",
    "prospects": DATA_DIR / "prospects.json"
}

def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

candidates_data = load_json(files["candidates"])
vacancies_data = load_json(files["vacancies"])
prospects_data = load_json(files["prospects"])

### Data overview

Displaying a summary of the first entries in each dataset

In [3]:
print("Candidates sample:\n", list(candidates_data.items())[:1])
print("\nVacancies sample:\n", list(vacancies_data.items())[:1])
print("\nProspects sample:\n", list(prospects_data.items())[:1])

Candidates sample:
 [('31000', {'infos_basicas': {'telefone_recado': '', 'telefone': '(11) 97048-2708', 'objetivo_profissional': '', 'data_criacao': '10-11-2021 07:29:49', 'inserido_por': 'Luna Correia', 'email': 'carolina_aparecida@gmail.com', 'local': '', 'sabendo_de_nos_por': '', 'data_atualizacao': '10-11-2021 07:29:49', 'codigo_profissional': '31000', 'nome': 'Carolina Aparecida'}, 'informacoes_pessoais': {'data_aceite': 'Cadastro anterior ao registro de aceite', 'nome': 'Carolina Aparecida', 'cpf': '', 'fonte_indicacao': ':', 'email': 'carolina_aparecida@gmail.com', 'email_secundario': '', 'data_nascimento': '0000-00-00', 'telefone_celular': '(11) 97048-2708', 'telefone_recado': '', 'sexo': '', 'estado_civil': '', 'pcd': '', 'endereco': '', 'skype': '', 'url_linkedin': '', 'facebook': ''}, 'informacoes_profissionais': {'titulo_profissional': '', 'area_atuacao': '', 'conhecimentos_tecnicos': '', 'certificacoes': '', 'outras_certificacoes': '', 'remuneracao': '', 'nivel_profissiona

### Initial exploration and cleaning

Data types and null values

In [4]:
def check_missing_values(df):
    return df.isnull().sum()

candidates_df = pd.json_normalize(candidates_data, sep='_')
vacancies_df = pd.json_normalize(vacancies_data, sep='_')
prospects_df = pd.json_normalize(prospects_data, sep='_')

print("Missing values in Candidates:\n", check_missing_values(candidates_df))
print("\nMissing values in Vacancies:\n", check_missing_values(vacancies_df))
print("\nMissing values in Prospects:\n", check_missing_values(prospects_df))

Missing values in Candidates:
 31000_infos_basicas_telefone_recado          0
31000_infos_basicas_telefone                 0
31000_infos_basicas_objetivo_profissional    0
31000_infos_basicas_data_criacao             0
31000_infos_basicas_inserido_por             0
                                            ..
5999_formacao_e_idiomas_nivel_ingles         0
5999_formacao_e_idiomas_nivel_espanhol       0
5999_formacao_e_idiomas_outro_idioma         0
5999_cv_pt                                   0
5999_cv_en                                   0
Length: 1738606, dtype: int64

Missing values in Vacancies:
 5185_informacoes_basicas_data_requicisao                     0
5185_informacoes_basicas_limite_esperado_para_contratacao    0
5185_informacoes_basicas_titulo_vaga                         0
5185_informacoes_basicas_vaga_sap                            0
5185_informacoes_basicas_cliente                             0
                                                            ..
12364_perfil_

### Detailed data pre-processing

Candidates (candidates.json)

In [5]:

def extract_ddd(phone):
    match = re.search(r'\(?(\d{2})\)?', phone or "")
    return match.group(1) if match else None

candidates_flat = []
for candidate_id, candidate_data in candidates_data.items():
    candidate_info = {
        'candidate_id': candidate_id,       
        'candidate_ddd_mobile': extract_ddd(candidate_data['informacoes_pessoais']['telefone_celular']),
        'candidate_pcd': candidate_data['informacoes_pessoais']['pcd'], 
        'candidate_certifications': candidate_data['informacoes_profissionais']['certificacoes'],        
        'candidate_academic_level': candidate_data['formacao_e_idiomas']['nivel_academico'],
        'candidate_english_level': candidate_data['formacao_e_idiomas']['nivel_ingles'],
        'candidate_spanish_level': candidate_data['formacao_e_idiomas']['nivel_espanhol'],
        }
    candidates_flat.append(candidate_info)
 
df_candidates = pd.DataFrame(candidates_flat)

In [6]:
df_candidates.sort_values(by='candidate_id', ascending=True, inplace=True)
df_candidates.replace(r'^\s*$', pd.NA, regex=True, inplace=True)

df_candidates['candidate_certifications'] = df_candidates['candidate_certifications'].apply(
    lambda x: 1 if pd.notna(x) and x != '' else 0
)

df_candidates['candidate_certifications'].value_counts()


candidate_certifications
0    41954
1      528
Name: count, dtype: int64

Prospects (prospects.json)

In [7]:
prospects_flat = []
for prospect_id, prospect_data in prospects_data.items():
    for prospect in prospect_data['prospects']:
        prospect_info = {
            'prospect_id': prospect_id,
            'prospect_candidate_code': prospect['codigo'],
            'prospect_candidate_status': prospect['situacao_candidado'],
            'prospect_application_date': prospect['data_candidatura'],
        }
        prospects_flat.append(prospect_info)

df_prospects = pd.DataFrame(prospects_flat)


In [8]:
df_prospects.sort_values(by='prospect_id', ascending=True, inplace=True)
df_prospects.head()

,prospect_id,prospect_candidate_code,prospect_candidate_status,prospect_application_date
13632,10,12611,Encaminhado ao Requisitante,07-12-2018
13624,10,12622,Encaminhado ao Requisitante,07-12-2018
13625,10,12621,Prospect,07-12-2018
13626,10,12620,Encaminhado ao Requisitante,07-12-2018
13627,10,12619,Encaminhado ao Requisitante,07-12-2018


Vacancies (vacancies.json)

In [9]:
vacancies_flat = []
for vacancy_id, vacancy_data in vacancies_data.items():
    vacancy_info = {
        'vacancy_id': vacancy_id,
        'vacancy_contract_type': vacancy_data['informacoes_basicas']['tipo_contratacao'],
        'vacancy_sap': vacancy_data['informacoes_basicas']['vaga_sap'],
        'vacancy_region': vacancy_data['perfil_vaga']['estado'],
        'vacancy_pcd': vacancy_data['perfil_vaga']['vaga_especifica_para_pcd'],
        'vacancy_professional_level': vacancy_data['perfil_vaga']['nivel profissional'],
        'vacancy_education_level': vacancy_data['perfil_vaga']['nivel_academico'],
        'vacancy_english_level': vacancy_data['perfil_vaga']['nivel_ingles'],
        'vacancy_spanish_level': vacancy_data['perfil_vaga']['nivel_espanhol'],
    }
    vacancies_flat.append(vacancy_info)


df_vacancies = pd.DataFrame(vacancies_flat)


In [10]:
df_vacancies.sort_values(by='vacancy_id', ascending=True, inplace=True)

df_vacancies['vacancy_contract_type'] = df_vacancies['vacancy_contract_type'].apply(lambda x: 'CLT' if isinstance(x, str) and 'CLT' in x else x)
df_vacancies['vacancy_contract_type'] = df_vacancies['vacancy_contract_type'].apply(lambda x: 'PJ' if isinstance(x, str) and 'PJ' in x else x)

df_vacancies.head(5)



,vacancy_id,vacancy_contract_type,vacancy_sap,vacancy_region,vacancy_pcd,vacancy_professional_level,vacancy_education_level,vacancy_english_level,vacancy_spanish_level
9642,10,CLT,Não,São Paulo,Não,Analista,Ensino Superior Incompleto,Técnico,Nenhum
9554,100,CLT,Não,São Paulo,Não,Analista,Ensino Superior Completo,Intermediário,Intermediário
12446,1000,CLT,Não,São Paulo,Não,Analista,Ensino Superior Completo,Básico,Básico
3176,10000,PJ,Não,São Paulo,Não,Sênior,Ensino Superior Completo,Básico,
3175,10001,CLT,Não,Minas Gerais,Não,Analista,Ensino Superior Completo,Básico,Básico


In [11]:
# 1. Merge entre vagas e prospecções (baseado em código do propscção )
#Utilização: Por exemplo, a vaga 10976 (chave no vacancies.json), possui 25 prospecções (chave 10976 no prospects.json), 
#onde o candidato “Sr. Thales Freitas”  (chave 41496 no applicants.json) foi contratado.

df_merged_1 = pd.merge(    
    df_vacancies,
    df_prospects,  
    left_on='vacancy_id',
    right_on='prospect_id',
    how='inner'
)

df_final = pd.merge(
    df_merged_1,
    df_candidates,
    left_on='prospect_candidate_code',
    right_on='candidate_id',
    how='inner'
)



In [12]:
df_final = df_final.applymap(lambda x: np.nan if x is None else x)
df_final.replace(r'^\s*$', pd.NA, regex=True, inplace=True)
df_final.fillna('Nao informado', inplace=True)

def group_status(status):
    if status in ['Prospect', 'Inscrito', 'Encaminhado ao Requisitante', 'Entrevista Técnica', 'Entrevista com Cliente', 'Em avaliação pelo RH']:
        return 'Em processo seletivo'
    elif status in ['Aprovado', 'Contratado pela Decision', 'Contratado como Hunting',
                    'Encaminhar Proposta', 'Proposta Aceita',
                    'Documentação PJ', 'Documentação CLT', 'Documentação Cooperado']:
        return 'Aprovado'
    elif status in ['Não Aprovado pelo Cliente', 'Não Aprovado pelo RH', 'Não Aprovado pelo Requisitante', 'Recusado']:
        return 'Reprovado'
    elif status in ['Desistiu', 'Desistiu da Contratação', 'Sem interesse nesta vaga']:
        return 'Desistiu'
    else:
        return 'Outro'

df_final['prospect_candidate_status'] = df_final['prospect_candidate_status'].apply(group_status)

df_final.drop(columns=['vacancy_id', 'prospect_id', 'prospect_candidate_code', 'candidate_id'], inplace=True)
df_final.head()


C:\Users\gabri\AppData\Local\Temp\ipykernel_8932\2565422392.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_final = df_final.applymap(lambda x: np.nan if x is None else x)


,vacancy_contract_type,vacancy_sap,vacancy_region,vacancy_pcd,vacancy_professional_level,vacancy_education_level,vacancy_english_level,vacancy_spanish_level,prospect_candidate_status,prospect_application_date,candidate_ddd_mobile,candidate_pcd,candidate_certifications,candidate_academic_level,candidate_english_level,candidate_spanish_level
0,CLT,Não,São Paulo,Não,Analista,Ensino Superior Incompleto,Técnico,Nenhum,Em processo seletivo,07-12-2018,11,Nao informado,0,Nao informado,Nao informado,Nao informado
1,CLT,Não,São Paulo,Não,Analista,Ensino Superior Incompleto,Técnico,Nenhum,Em processo seletivo,07-12-2018,11,Nao informado,0,Nao informado,Nao informado,Nao informado
2,CLT,Não,São Paulo,Não,Analista,Ensino Superior Incompleto,Técnico,Nenhum,Em processo seletivo,07-12-2018,11,Nao informado,0,Nao informado,Nao informado,Nao informado
3,CLT,Não,São Paulo,Não,Analista,Ensino Superior Incompleto,Técnico,Nenhum,Em processo seletivo,07-12-2018,11,Nao informado,0,Nao informado,Nao informado,Nao informado
4,CLT,Não,São Paulo,Não,Analista,Ensino Superior Incompleto,Técnico,Nenhum,Em processo seletivo,07-12-2018,11,Nao informado,0,Nao informado,Nao informado,Nao informado


### Saving Processed Data

In [13]:
PROCESSED_DIR = Path("../data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

df_candidates.to_csv(PROCESSED_DIR / "candidates.csv", index=False)
df_prospects.to_csv(PROCESSED_DIR / "prospects.csv", index=False)
df_vacancies.to_csv(PROCESSED_DIR / "vacancies.csv", index=False)
df_final.to_csv(PROCESSED_DIR / "df.csv", index=False)

print("Processed files saved in:", PROCESSED_DIR)

Processed files saved in: ..\data\processed
